# Single Run

In [1]:
import numpy as np
import pandas as pd
import pickle


from surgeryschedulingunderuncertainty.task import Task
from surgeryschedulingunderuncertainty.patients_provider import PatientsFromHistoricalDataProvider
from surgeryschedulingunderuncertainty.master import Master


from surgeryschedulingunderuncertainty.implementor import StandardImplementor
from surgeryschedulingunderuncertainty.optimizer import ImplementorAdversary


from surgeryschedulingunderuncertainty.predictive_model import NGBLogNormal, NGBNormal


np.random.seed(seed = 0)


/var/folders/w_/kqymfxcs5d561j515w6_cfqh0000gn/T/ipykernel_37389/3871698272.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Task: pazienti storici

In [2]:
task_file = open('task.pickle', 'rb')
     
# source, destination
task = pickle.load(task_file)                    
task_file.close()

or_master_schedule = pd.read_csv("../not_git/master_schedule_input_v2.csv", sep = ';')

task.master_schedule = Master(name = "my master", table=or_master_schedule)

In [3]:
or_master_schedule

,weekday,equipes,room,duration
0,1,C,or1,420
1,1,H,or2,420
2,1,D,or3,420
3,1,L,or4,420
4,1,F,or5,420
5,2,I,or1,420
6,2,C,or2,420
7,2,F,or3,420
8,2,B,or4,420
9,2,A,or5,420


In [4]:
len(task.patients)

300

### Implementor - Adversary

In [5]:
task.set_robustness_overtime=20 #10
task.set_robustness_risk = 0.1

In [6]:
implementor_adversary = ImplementorAdversary(task = task, 
                                             implementor=StandardImplementor(), 
                                             adversary=None, 
                                             description = 'Test')

In [7]:
implementor_adversary.create_instance()

In [8]:
# implementor_adversary._implementor.instance_data

In [9]:
schedule, report, solved_instance = implementor_adversary.run(15)

hello!
implementor


In [10]:
report

{'total duration': 0,
 'start time': datetime.datetime(2024, 8, 4, 13, 5, 13, 271106),
 'end time': datetime.datetime(2024, 8, 4, 13, 16, 30, 541927),
 'number of iterations': 1,
 'is last robust': 'Yes',
 'iterations': [{'flag': True, 'duration': 11.287846016666666}]}

In [11]:
schedule

{'task description': 'My first problem',
 'creation date': '2024-08-04 13:16:30',
 'number of blocks': 75,
 'blocks': [{'patients ids': [7545, 6443, 4212, 2898, 2547],
   'patients durations (nominal)': [88.42271413692872,
    90.8013349464056,
    78.0816445545342,
    74.35961340805622,
    78.08173888420185],
   'patients urgencies': [3, 4, 4, 3, 4],
   'patients max waiting days': [180, 360, 360, 180, 360],
   'patiens delay in days': [0, 0, 0, 0, 0],
   'patient equipe': ['C', 'C', 'C', 'C', 'C'],
   'total duration (nominal)': 409.74704593012655,
   'block allocated duration': 420,
   'room': 'or1',
   'weekday': 1,
   'order_in_day': 1,
   'week': 0,
   'days since beginning': 0,
   'equipes': ['C'],
   'overtime probability': 0},
  {'patients ids': [945, 7246, 619, 2865, 4459],
   'patients durations (nominal)': [102.22446359150945,
    73.24526871967406,
    100.42906469471428,
    58.35180944108222,
    85.37185776235913],
   'patients urgencies': [4, 4, 4, 4, 4],
   'patient

In [12]:
#                    if solved_instance.x[block_index+1, num_pat+1]() == 1:

sum(solved_instance.x[:,:]())


220.0

In [13]:
[x.get('patients ids') for x in schedule.get('blocks')]

[[7545, 6443, 4212, 2898, 2547],
 [945, 7246, 619, 2865, 4459],
 [4591, 7113, 7848, 6293, 5027, 2243, 2976, 8018],
 [937, 1898, 1860],
 [2348, 3237, 1185, 359],
 [290, 1706],
 [557, 4520, 7022, 8323],
 [2523, 7038, 5803, 9127, 7999, 5895, 2663],
 [649, 5375, 3408, 967, 7989],
 [8200, 4290, 4324, 5808],
 [7569, 2187],
 [6266, 2881],
 [3921, 1715, 3132, 4848, 4361, 8475, 1924],
 [5714],
 [2650, 7678, 4761],
 [8026, 7357],
 [1158, 1850, 1251],
 [6875, 6932, 5940, 8937, 7717],
 [3786, 6580, 7520, 4235],
 [1335, 2517, 6777, 6855],
 [3835],
 [4662, 6411, 7914],
 [5440, 6710, 6667, 7219, 5844],
 [6298, 4990, 8904],
 [8516, 4644, 5214, 7988],
 [5539, 640, 568, 880],
 [5590, 7929, 1417, 6210],
 [5093, 6335, 9023, 3167, 6903, 6344, 3273, 6284, 6604],
 [5436, 4483],
 [5137, 8290, 5705, 2261, 6822, 7017],
 [8584, 7858],
 [4999, 175],
 [6014, 5713, 7530, 4298, 3505, 3283, 4779],
 [3818],
 [],
 [3774, 8614],
 [1107, 2740],
 [8218, 7044, 6935, 8889, 550, 2706, 1039],
 [4985, 6614],
 [],
 [7083],
 [10

In [29]:
len(schedule.get('patients not included equipe'))

236

In [11]:
from collections import Counter
Counter(schedule.get('patients not included equipe'))

Counter({'L': 45,
         'H': 43,
         'F': 39,
         'D': 37,
         'G': 25,
         'I': 18,
         'C': 13,
         'E': 7,
         'B': 6,
         'A': 3})

solution._blocks[4].patients[0].uncertainty_profile.param_s

In [12]:
equipes = []
for patient in task.patients:
    equipes.append(patient.equipe)
Counter(equipes)

Counter({'L': 51,
         'F': 50,
         'D': 49,
         'H': 48,
         'G': 25,
         'I': 24,
         'C': 22,
         'B': 14,
         'A': 10,
         'E': 7})

### Chance constraint

In [ ]:
from surgeryschedulingunderuncertainty.optimizer import VanillaImplementor
from surgeryschedulingunderuncertainty.implementor import ChanceConstraintsImplementor


In [ ]:
chance_constraints = VanillaImplementor(task = task, 
                                              implementor=ChanceConstraintsImplementor(task = task), 
                                              description = 'Test')

In [ ]:
chance_constraints.create_instance()

In [ ]:
solution_chance_constraints = chance_constraints.run()

In [ ]:
for pat in (solution_chance_constraints._blocks[0].patients):
    print(pat)

### Budget Set

In [ ]:
from surgeryschedulingunderuncertainty.optimizer import  BudgetSet
from surgeryschedulingunderuncertainty.implementor import  BudgetSetImplementor


In [ ]:
task.gamma_variation = 0


budget_set = BudgetSet(task = task, 
                              implementor=BudgetSetImplementor(task = task), 
                              description = 'Test')






In [ ]:
soulution_budget_set = budget_set.run()

In [ ]:
budget_set._instance.get(None).get('time_increment')

In [ ]:
budget_set._instance.get(None).get('gamma')

In [ ]:
for block in soulution_budget_set._blocks:
    print(block)
    for pat in (block.patients):
        print(pat)
    print('---')